In [1]:
# indir = "/data/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/vireo/nclones20/annotation_clones/DE_large"
# outdir = "/data/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/vireo/nclones20/annotation_clones/DE_large/minPct_0.01__logThresh_0.1"
# min_pct = 0.01
# logfc_thresh = 0.1 
# n_donors = 4

indir = "/data/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/annotation_clones/DE_large"
outdir = "/data/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/annotation_clones/DE_large/minPct_0__logThresh_0"
min_pct = 0
logfc_thresh = 0
n_donors = 4

# If more than 2 clones pass the cdf_thresh, use that, o.w use n_top_clones
cdf_thresh = 0.5
n_top_clones = 3

# min_pct = 0
# logfc_threshold = 0.1
use_latent_vars = FALSE
genome = "/data/Mito_Trace/data/external/GRCh38_MT_blacklist/fasta/genome.fa"
cores=16
utils_script <- "/data/Mito_Trace/R_scripts/seurat_utils.R"





In [2]:
if (use_latent_vars){
    latent.vars = "peaks_depth"
}else{
    latent.vars=NULL
}

In [3]:
source(utils_script)

library(Seurat)
library(Signac)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(magrittr)
library(cowplot)
library(metap)
library(dplyr)
library(future)
plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)
library("ComplexHeatmap")

Attaching SeuratObject


Attaching package: ‘cowplot’


The following object is masked from ‘package:patchwork’:

    align_plots



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'.”
Loading required package: grid

ComplexHeatmap version 2.10.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.

The new InteractiveComplexHeatmap package

## Import seurat object and clones

In [4]:
large.clones <- read.csv(file=file.path(indir, "large_clones.csv"))
se <- readRDS(file=file.path(indir, "se.clonesfilt.rds"))
Idents(se) <- se$name
head(Idents(se))
             

Control_AAACGAAAGAGGTCCA-1 Control_AAACGAAAGCGATACG-1 
                      3_53                        1_6 
Control_AAACGAACAATAGTGA-1 Control_AAACGAACACAATAAG-1 
                      3_47                       0_54 
Control_AAACGAACATCGTGAT-1 Control_AAACGAAGTAACGGTG-1 
                       3_0                       3_53 
27 Levels: 0_0 0_1 0_2 0_54 1_0 1_1 1_14 1_2 1_3 1_5 1_6 1_7 1_9 2_0 ... 3_53

## Run DE

In [6]:
curr.outdir <- file.path(outdir, paste0("all.Donors"))
dir.create(curr.outdir)


In [7]:
            
#donor.large.clones <- large.clones %>% filter(donor==d)
clones.filt.ids <- sort(large.clones$name)
print('clones')
print(clones.filt.ids)
print(length(clones.filt.ids))
#se <- subset(se, subset = (donor==d) & (lineage %in% large.clones$lineage))
#Idents(se) <- se$lineage

print('se')
print(dim(se[[]]))

pairs = combn(clones.filt.ids,2)

[1] "clones"
 [1] "0_0"  "0_1"  "0_2"  "0_54" "1_0"  "1_1"  "1_14" "1_2"  "1_3"  "1_5" 
[11] "1_6"  "1_7"  "1_9"  "2_0"  "2_1"  "2_10" "2_11" "2_2"  "2_3"  "2_4" 
[21] "2_5"  "2_6"  "3_0"  "3_1"  "3_2"  "3_47" "3_53"
[1] 27
[1] "se"
[1] 5357   39


In [9]:

for (i in 1:dim(combn(clones.filt.ids,2))[2]){
    DefaultAssay(se) <- "chromvarnames"
    print(pairs[,i])
    try({
        print(paste("clones", i))
        a = pairs[1,i]
        b = pairs[2,i]
        da <- FindMarkers(
          object = se,
          ident.1 = a,
          ident.2 = b,
          only.pos = FALSE,
          mean.fxn = rowMeans,
          logfc.threshold = logfc_thresh,
          min.pct = min_pct,
          #latent.vars=latent.vars,
          fc.name = "avg_diff")

        da$p_val_adj_BH <- stats::p.adjust(da$p_val, method = "BH", n = length(da$p_val))
        write.csv(da, 
                  file=file.path(curr.outdir, paste0("clones_",a,"__", b,".DE.TF.csv")), quote=F)
        #plotDE(se, da, c, clust_outdir)

        gally <- GGally::ggpairs(da[,c("p_val", "p_val_adj", "avg_diff", "p_val_adj_BH" )], aes(alpha = 0.4))
        ggsave(plot=gally, file=file.path(curr.outdir, paste0("clones_",a,"__", b,".DE.TF.pvalHist.png")))
        # Save motifs
        DefaultAssay(se) <- "ATAC"
        raw.ids <- names(Motifs(se)@motif.names)
        print(head(rownames(da)))
        rownames(da) <- sapply((rownames(da)), function(x){raw.ids[match(x, Motifs(se)@motif.names)]})
        m <- MotifPlot(
          object = se,
          motifs = head(rownames(da)),
          assay = 'ATAC'
        )
        ggsave(plot=m, 
                   file=file.path(curr.outdir, paste0("clones_",a,"__", b,".DE.TF.motif.top.png")))

    })
}


[1] "0_0" "0_1"
[1] "clones 1"


Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2

Saving 6.67 x 6.67 in image



[1] "JUN::JUNB"   "FOSL2"       "FOSL2::JUND" "FOS::JUND"   "FOSL1::JUN" 
[6] "FOSL2::JUNB"


Warning message:
“`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead.”
Saving 6.67 x 6.67 in image



[1] "0_0" "0_2"
[1] "clones 2"


...

Warning message:
“`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead.”
Saving 6.67 x 6.67 in image



[1] "3_47" "3_53"
[1] "clones 351"


Saving 6.67 x 6.67 in image



[1] "ONECUT3"    "ONECUT1"    "ONECUT2"    "FOSL2"      "FOSL2::JUN"
[6] "SCRT2"     


Warning message:
“`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead.”
Saving 6.67 x 6.67 in image



In [ ]:
# for (d in 0:(n_donors-1)){
#     print('donor')
#     print(d)
#     curr.outdir <- file.path(outdir, paste0("donor", d, "_TF"))
#     dir.create(curr.outdir)
#     donor.large.clones <- large.clones %>% filter(donor==d)
#     clones.filt.ids <- sort(donor.large.clones$lineage)
#     print('clones')
#     print(clones.filt.ids)
#     print(length(clones.filt.ids))
#     se.filt <- subset(se, subset = (donor==d) & (lineage %in% donor.large.clones$lineage))
#     Idents(se.filt) <- se.filt$lineage  
#     print('se.filt')
#     print(dim(se.filt[[]]))

#     pairs = combn(clones.filt.ids,2)
#     for (i in 1:dim(combn(clones.filt.ids,2))[2]){
#         DefaultAssay(se.filt) <- "chromvarnames"
#         print(pairs[,i])
#         try({
#             print(paste("clones", i))
#             a = pairs[1,i]
#             b = pairs[2,i]
#             da <- FindMarkers(
#               object = se.filt,
#               ident.1 = a,
#               ident.2 = b,
#               only.pos = FALSE,
#               mean.fxn = rowMeans,
#               logfc.threshold = logfc_thresh,
#               min.pct = min_pct,
#               #latent.vars=latent.vars,
#               fc.name = "avg_diff")
            
#             da$p_val_adj_BH <- stats::p.adjust(da$p_val, method = "BH", n = length(da$p_val))
#             write.csv(da, 
#                       file=file.path(curr.outdir, paste0("clones_",a,"__", b,".DE.TF.csv")), quote=F)
#             #plotDE(se, da, c, clust_outdir)
    
#             gally <- GGally::ggpairs(da[,c("p_val", "p_val_adj", "avg_diff", "p_val_adj_BH" )], aes(alpha = 0.4))
#             ggsave(plot=gally, file=file.path(curr.outdir, paste0("clones_",a,"__", b,".DE.TF.pvalHist.png")))
#             # Save motifs
#             DefaultAssay(se.filt) <- "ATAC"
#             raw.ids <- names(Motifs(se.filt)@motif.names)
#             print(head(rownames(da)))
#             rownames(da) <- sapply((rownames(da)), function(x){raw.ids[match(x, Motifs(se.filt)@motif.names)]})
#             m <- MotifPlot(
#               object = se.filt,
#               motifs = head(rownames(da)),
#               assay = 'ATAC'
#             )
#             ggsave(plot=m, 
#                        file=file.path(curr.outdir, paste0("clones_",a,"__", b,".DE.TF.motif.top.png")))

#         })
#     }
# }